In [2]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn 
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from utils.dataset import MIND_Train
from utils.manager import Manager
from utils.util import load_pickle, save_pickle

In [3]:
class config:
    epochs = 10
    scale = "demo"
    mode = "train"
    device = 0
    batch_size = 2
    batch_size_encode = 2
    dropout_p = 0.1
    seed = 3407
    world_size = 1

    data_root = "../../../Data"
    cache_root = "data/cache"
    dataloaders = ["train", "dev", "news", "history"]

    his_size = 50
    impr_size = 20
    negative_num = 4

    max_title_length = 64
    max_abs_length = 256
    title_length = 32
    abs_length = 64

    plm = "bert"

    enable_fields = ["title", "abs"]
    enable_gate = "weight"

    rank = 0
    verbose = None
    distributed = False
    debug = False

manager = Manager(config, notebook=True)
loaders = manager.prepare()

[2022-02-10 11:44:52,107] INFO (Manager) Hyper Parameters are:
{'scale': 'demo', 'batch_size': 2, 'batch_size_encode': 2, 'dropout_p': 0.1, 'seed': 3407, 'dataloaders': ['train', 'dev', 'news', 'history'], 'his_size': 50, 'impr_size': 20, 'negative_num': 4, 'title_length': 32, 'abs_length': 64, 'plm': 'bert', 'enable_fields': ['title', 'abs'], 'enable_gate': 'weight', 'verbose': None, 'sequence_length': 96}

[2022-02-10 11:44:52,110] INFO (MIND_Train) Loading Cache at MINDdemo_train
[2022-02-10 11:44:53,693] INFO (MIND_Dev) Loading Cache at MINDdemo_dev
[2022-02-10 11:44:54,852] INFO (MIND_News) Loading Cache at MINDdemo_dev


In [6]:
t = AutoTokenizer.from_pretrained(manager.plm_dir)
m = AutoModel.from_pretrained(manager.plm_dir).to(0)

In [10]:
a = torch.rand(2,3,768,device=0)
m(inputs_embeds=a)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1178,  0.1205,  0.0734,  ..., -0.3714, -0.1969,  0.1553],
         [ 0.4199,  0.4311, -0.5809,  ..., -0.6447,  0.0505,  0.1728],
         [ 0.1379,  0.1555, -0.0594,  ..., -0.4050, -0.1968,  0.2712]],

        [[ 0.2416,  0.2850,  0.2549,  ..., -0.1965, -0.3362, -0.7308],
         [ 0.3608, -0.0789,  0.0028,  ..., -0.1577, -0.0663, -0.5939],
         [ 0.4457, -0.1945,  0.0065,  ...,  0.0430, -0.0462, -0.7706]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-0.7482, -0.0253,  0.9886,  ...,  0.8083, -0.3505,  0.8934],
        [-0.5025, -0.5396,  0.1007,  ..., -0.6285, -0.1406,  0.3769]],
       device='cuda:0', grad_fn=<TanhBackward>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

In [12]:
a = m.embeddings.word_embeddings

In [13]:
m.embeddings.word_embeddings = None

In [20]:
m.get_extended_attention_mask(torch.tensor([[1, 1, 0], [1, 0, 0]]), torch.Size((2,3)), "cpu").shape

torch.Size([2, 1, 1, 3])

In [24]:
from models.modules.attention import extend_attention_mask
extend_attention_mask(torch.tensor([[1, 1, 0], [1, 0, 0]])).shape

torch.Size([2, 1, 1, 3])

In [4]:
loader_train = loaders["train"]
loader_dev = loaders["dev"]
loader_news = loaders["news"]

dataset_train = loader_train.dataset
dataset_dev = loader_dev.dataset
dataset_news = loader_news.dataset

X1 = iter(loader_train)
X2 = iter(loader_dev)
X3 = iter(loader_news)
x = next(X1)
x2 = next(X2)
x3 = next(X3)

In [34]:
# check train loader result
nid2index = load_pickle("/data/v-pezhang/Code/GateFormer/src/data/cache/MIND/MINDdemo_train/news/nid2index.pkl")
uid2index = load_pickle("/data/v-pezhang/Code/GateFormer/src/data/cache/MIND/uid2index.pkl")
nindex2id = {v:k for k,v in nid2index.items()}
uindex2id = {v:k for k,v in uid2index.items()}

# check behaviors.tsv
print([uindex2id[i] for i in x["user_index"].tolist()], (x["impr_index"] + 1).tolist())
# check news.tsv
print([nindex2id[i] for i in x["cdd_idx"][0][:5].tolist()])
print(t.batch_decode(x["cdd_token_id"][0][:5], skip_special_tokens=True))

print([nindex2id[i] for i in x["his_idx"][0][:5].tolist()])
print(t.batch_decode(x["his_token_id"][0][:5], skip_special_tokens=True))

['U63949', 'U28675'] [1633, 526]
['N33159', 'N29128', 'N55689', 'N59981', 'N47229']
["sheriff's deputy charged with murder after man shot, killed a sheriff's deputy is in jail, charged with murder after a deadly shooting in athens, authorities said.", "flight attendants have a secret language you didn't know about here are some phrases only flight attendant use and what they actually mean.", 'charles rogers, former michigan state football, detroit lions star, dead at 38 charles rogers, the former michigan state football star whom the detroit lions selected with the second overall pick in 2003 nfl draft, has died.', "tamron hall talks leaving the today show, jokes about megyn kelly's reported multimillion payout tamron hall talks losing the today show", 'these are the best harley - davidson motorcycles you can get for the cheap these are the best harleys to buy right now.']
['N1150', 'N29177', 'N29898', 'N46392', 'N54772']
['a texas mom is going to prison after putting her son through u

In [33]:
# check dev loader result
nid2index = load_pickle("/data/v-pezhang/Code/GateFormer/src/data/cache/MIND/MINDdemo_dev/news/nid2index.pkl")
uid2index = load_pickle("/data/v-pezhang/Code/GateFormer/src/data/cache/MIND/uid2index.pkl")
nindex2id = {v:k for k,v in nid2index.items()}
uindex2id = {v:k for k,v in uid2index.items()}

# check behaviors.tsv
print([uindex2id[i] for i in x2["user_index"].tolist()], (x2["impr_index"] + 1).tolist())
# check news.tsv
print([nindex2id[i] for i in x2["cdd_idx"][0][:5].tolist()])
print(t.batch_decode(x2["cdd_token_id"][0][:5], skip_special_tokens=True))

print([nindex2id[i] for i in x2["his_idx"][0][:5].tolist()])
print(t.batch_decode(x2["his_token_id"][0][:5], skip_special_tokens=True))

['U80234', 'U80234'] [1, 1]
['N28682', 'N48740', 'N31958', 'N34130', 'N6916']
["browns apologize to mason rudolph, call myles garrett's actions'unacceptable'players often defend their teammates when commenting on an incident like the one we saw thursday, but the immediate reaction from mayfield and others proves just how far garrett crossed the line.", "i've been writing about tiny homes for a year and finally spent 2 nights in a 300 - foot home to see what it's all about i stayed in a tiny house for three days to see what the fuss was all about, and i was surprised by what i saw.", 'opinion : colin kaepernick is about to get what he deserves : a chance the end may be near for the 3 - year - old saga of colin kaepernick as the quarterback is scheduled to work out for teams on saturday.', "the kardashians face backlash over'insensitive'family food fight in kuwtk clip kardashian's face backlash over family food fight", "then and now : what all your favorite'90s stars are doing today thes